In [ ]:
%load_ext nb_black

In [3]:
from random import randint
import numpy as np
import random
from numpy import array
from numpy import argmax
from numpy import array_equal
from tensorflow.keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense

In [4]:
def random_dna_sequence(length):
    return "".join(random.choice("ACTG") for _ in range(length))


def base_frequency(dna):
    d = {}
    for base in "ATCG":
        d[base] = dna.count(base) / float(len(dna))
    return d


data = []
for _ in range(100):
    dna = random_dna_sequence(100)
    data.append(dna)

In [5]:
data

['CACCGCCATCAGGTTATAGCTGCAATTTCATACAACTATTCCGGCGAGTCCGAGCTGGCACCACTGATCTAGTTCGTTAATGTACCGCATTCGTTACTAA',
 'GACTTTGGGGTTTGATTGCGCATCCAACGTACAATCCAGTGTCCCTGGCTGAGGCGATCCGATGTCAGCTTTGCCGATGCAACGGACCACGACGTTTAAT',
 'TAGGAACGCACTCATTGAATGGCGCACCCAACGAGGGCGGTTGGCGGCCTCTATCCGATGGATCTGCGGGCCGGTATATCCAACTCCGATTACGTATTCA',
 'TCGGGGGCCTTTCTTAGAAAAAGTCAGGATATCACCCACATCTCACGCGAAGATCCGGCAAGTTCAGACCAGTGCCCACACGAAGCAAGCACCGCTGACG',
 'TATGACCCCGCCCCTAAGCAGGGACGAAAGATGTCCGGCTCCTCTCAACTATAGTGAAGGTCGTGGAGAATCGACGTAACGATTAATAGTGTCGGAGATT',
 'CATCGCAGGCGCATCTAAGACATGTTATTAGACGAATGGTAGGAAAATATAACCTGGATCCACCTGCACGTTTACACGGCGGTTCAGGTGCCCATACTCA',
 'AGTATAATGACGGTGATTACGCAGAACGACACAGAAGCGGGGCCGCTTGGAGTGCTCGGTTGCGCTGAGTATCCCAATGTGGCAAGAGCCATAACTCACG',
 'ACCCATTACACCTATTCCCCATCGGTAGACATAGGCAATAACTTGTTGTCTCCGGGAAGCCCTTCTCGACACGCCACCAAAGGGAGCACGTGCCCCGTAG',
 'GTGCAATAAACCCTGGCACGTGTGGAATAATAAGTGACCAATTGTTGTTAGAAATCTTGTCAGGTCTTCGCCGACTGACCGCCACGTTCTAACCAAGTAG',
 'AGCCGCCCATGCGTCCAAATTTGCAGGCCCTTCTCTAGAACAAAGCAAAACGT

In [16]:
import numpy as np

# seqs = ["GAATTCTCGAA", "AAATTCTCGAA"]

CHARS = "ACGT"
CHARS_COUNT = len(CHARS)
print(CHARS_COUNT)

maxlen = max(map(len, data))
print(maxlen)
res = np.zeros((len(data), CHARS_COUNT * maxlen))



4
100


In [17]:
res.shape

(100, 400)

In [14]:
for si, seq in enumerate(data):
    seqlen = len(seq)
    arr = np.chararray((seqlen,), buffer=seq)
    for ii, char in enumerate(CHARS):
        res[si][ii * seqlen : (ii + 1) * seqlen][arr == char] = 1


In [15]:
res[0]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [ ]:
# generate a sequence of random integers
def generate_sequence(length, n_unique):
    return [randint(1, n_unique - 1) for _ in range(length)]

In [ ]:
# prepare data for the LSTM
def get_dataset(n_in, n_out, cardinality, n_samples):
    X1, X2, y = list(), list(), list()
    for _ in range(n_samples):
        # generate source sequence
        source = generate_sequence(n_in, cardinality)
        # define padded target sequence
        target = source[:n_out]
        target.reverse()
        # create padded input target sequence
        target_in = [0] + target[:-1]
        # encode
        src_encoded = to_categorical([source], num_classes=cardinality)
        tar_encoded = to_categorical([target], num_classes=cardinality)
        tar2_encoded = to_categorical([target_in], num_classes=cardinality)
        # store
        X1.append(src_encoded)
        X2.append(tar2_encoded)
        y.append(tar_encoded)
    return array(X1), array(X2), array(y)

In [ ]:
# returns train, inference_encoder and inference_decoder models
def define_models(n_input, n_output, n_units):
	# define training encoder
	encoder_inputs = Input(shape=(None, n_input))
	encoder = LSTM(n_units, return_state=True)
	encoder_outputs, state_h, state_c = encoder(encoder_inputs)
	encoder_states = [state_h, state_c]
	# define training decoder
	decoder_inputs = Input(shape=(None, n_output))
	decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
	decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
	decoder_dense = Dense(n_output, activation='softmax')
	decoder_outputs = decoder_dense(decoder_outputs)
	model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
	# define inference encoder
	encoder_model = Model(encoder_inputs, encoder_states)
	# define inference decoder
	decoder_state_input_h = Input(shape=(n_units,))
	decoder_state_input_c = Input(shape=(n_units,))
	decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
	decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
	decoder_states = [state_h, state_c]
	decoder_outputs = decoder_dense(decoder_outputs)
	decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
	# return all models
	return model, encoder_model, decoder_model


In [ ]:
# generate target given source sequence
def predict_sequence(infenc, infdec, source, n_steps, cardinality):
    # encode
    state = infenc.predict(source)
    # start of sequence input
    target_seq = array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
    # collect predictions
    output = list()
    for t in range(n_steps):
        # predict next char
        yhat, h, c = infdec.predict([target_seq] + state)
        # store prediction
        output.append(yhat[0, 0, :])
        # update state
        state = [h, c]
        # update target sequence
        target_seq = yhat
    return array(output)

In [ ]:
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
    return [argmax(vector) for vector in encoded_seq]

In [ ]:
# configure problem
n_features = 50 + 1
n_steps_in = 6
n_steps_out = 3

In [ ]:
# define model
train, infenc, infdec = define_models(n_features, n_features, 128)
train.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# generate training dataset
X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 100000)
print(X1.shape, X2.shape, y.shape)


In [ ]:
# train model
train.fit([X1, X2], y, epochs=50)

In [ ]:
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=32, epochs=10, verbose=1)

# Get training and test loss histories
training_loss = history.history['loss']
test_loss = history.history['val_loss']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show();

In [ ]:
# evaluate LSTM
total, correct = 100, 0
for _ in range(total):
	X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 1)
	target = predict_sequence(infenc, infdec, X1, n_steps_out, n_features)
	if array_equal(one_hot_decode(y[0]), one_hot_decode(target)):
		correct += 1
print('Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))
# spot check some examples
for _ in range(10):
	X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 1)
	target = predict_sequence(infenc, infdec, X1, n_steps_out, n_features)
	print('X=%s y=%s, yhat=%s' % (one_hot_decode(X1[0]), one_hot_decode(y[0]), one_hot_decode(target)))

In [ ]:
# generate a sequence of random integers
def generate_sequence(length, n_unique):
    return [randint(1, n_unique - 1) for _ in range(length)]

In [ ]:
def get_dataset(n_in, n_out, cardinality, n_samples):
    X1, X2, y = list(), list(), list()
    for _ in range(n_samples):
        # generate source sequence
        source = generate_sequence(n_in, cardinality)
        # define padded target sequence
        target = source[:n_out]
        target.reverse()
        # create padded input target sequence
        target_in = [0] + target[:-1]
        # encode
        src_encoded = to_categorical([source], num_classes=cardinality)
        tar_encoded = to_categorical([target], num_classes=cardinality)
        tar2_encoded = to_categorical([target_in], num_classes=cardinality)
        # store
        X1.append(src_encoded)
        X2.append(tar2_encoded)
        y.append(tar_encoded)
    return array(X1), array(X2), array(y)

In [ ]:
X1[0]

In [ ]:
X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 100000)

In [ ]:
# configure problem
n_features = 51
n_steps_in = 6
n_steps_out = 3

In [ ]:
m, n = y.shape[::2]
y = y.transpose(0, 3, 1, 2).reshape(m, n, -1)

In [ ]:
m

In [ ]:
n

In [ ]:
data_new.shape

In [ ]:
# generate training dataset

print(X1.shape, X2.shape, y.shape)
